In [17]:
## READING IN PACKAGES
import pandas as pd
import pyodbc
import adodbapi
import inspect 
import plotly.graph_objects as go
import os
import numpy as np
from datetime import date

INPUT VARIABLES IN CODE CHUNK BELOW

In [18]:
### --- DEFINE VARIABLES ----

## Block Models

## EBS Block Model (EBS)
EBSBlockModel =  "waebs_DepA_20220928_002" 

## Regularised Block Model (RBM)
RegBlockModel =  "waapr2_DepA_20220928_021"

## ORR/ORS Model - leave blank ("") if no ORR Model
ORR_ORSModel =  "" 

## ModFactor Table - only used if OOR_ORSModel = ""
MF_table = [['hg', 's_l_t + s_f_t',1],
            ['hga', 's_l_t + s_f_t',1],
            ['lga', 's_l_t + s_f_t',0],
            ['lgs', 's_l_t + s_f_t',0],
            #['lgo', 's_l_t + s_f_t',0],
            ['w', 's_l_t + s_f_t',0] 
            ]

ModFactor = pd.DataFrame(MF_table, columns = ['Field', 'plant', 'Ratio'])


## Other variable inputs

## Resource Cutoff
RCut =  't_fe>50'#"dest_tag in ('hg','bpo')" 
RCutGroup = "dest_tag"
 

## Mining cutoff (FCOG)
MCut  = "t_fe>=58"
MCutGroup = "mp_dest"

## LOM Schedule Cutoff (VCOG)
SCut = "t_fe>=58"
SCutGroup = "mp_dest"

## Topo Filter
topo = "topo>202208" 

## filled Filter
filled = ""

## Mined out topo 
topoMined = "topo>99998"

## topo check
topoCheck = "Not identical"

## inut
inut = 'mp_dest'

## mp_parcel
mp_parcel = 't_fe>50' 

#Adjustments for Waterfall Table (default 0)
adjustment_1 = 0
adjustment_2 = 0

adjustment_3 = 0

In [19]:
## -------------------------------------------------------------------Create Connection to Database--------------------------------------------------------
def databaseConnection(provider, dataSource, dataBase, connection):
    """ 
     returns adodbapi.adodbapi.Connection type

    :param provider: database provider
    :param dataSource: Data Source/Server
    :param dataBase: Database
    :param connection: Trusted Connection type (Yes/no)
    """
    print("Creating database connection")
    connection  = adodbapi.connect(f"""
                                       PROVIDER={provider};
                                       Data Source={dataSource};
                                       Database={dataBase};
                                       trusted_connection={connection};
                                    """)
    print("Connection Successfull")
    return connection

## -------------------------------------------------------------------Check Topo Query--------------------------------------------------------

def checkTopo(EBS, RBM, connection):
    """ 
     checks the topo of EBS and RBM

    :param EBS: EBS Block Model
    :param RBM: Regularised Block Model
    """
    ## Query for selecting data from EBS table
       
    ebsQuery = f"""
                SELECT max(topo) from {EBS}
                WHERE topo != 999999
                """

    print(f"Running query: {ebsQuery} ")
    
    ebsData = pd.read_sql(ebsQuery,connection)

    if not ebsData.empty:
        topoEbs = ebsData.iloc[0][0]
    else:
        topoEbs = False

    ## -------------------------------------------------------------------
    
     ## Query for selecting data from RBM table

    
    rbmQuery = f"""
                SELECT max(topo) from {RBM}
                WHERE topo != 999999
                """
    print(f"Running query: {rbmQuery} ")

    rbmData = pd.read_sql(rbmQuery,connection)

    if not rbmData.empty:
            topoRbm = rbmData.iloc[0][0]
    else:
        topoRbm = False       
    
    return topoEbs, topoRbm

## -------------------------------------------------------------------Variable check--------------------------------------------------------

def variableChecks(RCut,
                   MCut,
                   EBSBlockModel,
                   RegBlockModel,
                   ORR_ORSModel,
                   topo,
                   filled,
                   topoMined
                   ):
    """
    Function to check variables for content
    """

    ## Check RCut
    if RCut == "":
        RCut = "reportable in ('resource', 'to_reserve')"
    else:
        RCut = f"{RCut} and reportable in ('resource', 'to_reserve')"

    ## Check MCut
    if MCut == "":
        RCut =  "reportable in ('resource', 'to_reserve')" ## VBA has this as RCut even though we are check MCut variable - not sure if mistake - follow up with Mike
    else:
        MCut = f"{MCut} and reportable in ('resource', 'to_reserve')"

    ## Check EBS
    if EBSBlockModel == "":
        print(f"Please check the input for EBSBlockModel, it is currently empty")
        #raise ValueError("Invalid input for EBSBlockModel") ## not going to raise error until main
    
    ## Check RBModel
    if RegBlockModel == "":
        print("Error: Sublock model input not recorded")
        raise ValueError("Invalid input for RegBlockModel")

   
    #if ORR_ORSModel == "":
       #raise ValueError("Invalid input for ORR_ORSModel")

    ## Check top0
    if topo == "":
        topo = "topo > 0"
    
    ## Check filled
    if filled == "":
        filled = "filled = 999999"
    
    ## Check topomined
    if topoMined == "":
        topoMined = "topo = 999999"

    
    ## If passes all checks then return True
    return RCut, MCut, EBSBlockModel, RegBlockModel, ORR_ORSModel, topo, filled, topoMined, True
    
## -------------------------------------------------------------------Query function--------------------------------------------------------

def queryFunction(connection,
                  step,
                  query):
    
    # query = f"""
    #          SELECT {target}, sum(t_t)
    #          FROM {EBS}
    #          WHERE fe_bin NOT IN('00.0','null')
    #          AND {topo}
    #          AND {filled}
    #          {additional}
    #          GROUP BY {target}
    #          """
    print("\n" * 3)
    print("*" * 50)
    print(f"""Running {step} query:""")
    print(inspect.cleandoc(query))
    print("*" * 50)

    result = pd.read_sql(query, connection)

    
    print(f"{step} query complete")

    if result.empty:
        print(f"{step} query returned an empty dataset, please check variable inputs")
    else:
        print(f"{step} query returned a non-empty dataset, program will continue")
    return result 

## -------------------------------------------------------------------finding the lease names and separating exploration lease function--------------------------------------------------------
def findLeaseName(leaseNamesTable): ## are we just looking for
    """
    Finding the lease names and separating exploration lease.
    """
    ## find records in leaseNamesTable where the record DOES NOT start with "m"
    explorationLeases = leaseNamesTable[~leaseNamesTable["lease_name"].str.startswith("m","a")]["lease_name"].reset_index(drop = True)

    explorationLeasesString = ""
    for i in range(len(explorationLeases)):
        if i == 0:
            explorationLeasesString = f"'{explorationLeases[i]}'"
        else:
            explorationLeasesString += f", '{explorationLeases[i]}'"
    
    if explorationLeasesString != "":
        explorationLeasesString = f"AND lease_name NOT IN ({explorationLeasesString})"

    return explorationLeasesString


In [20]:
def main(EBSBlockModel,
         RegBlockModel,
         ORR_ORSModel,
         RCut,
         RCutGroup,
         MCut,
         MCutGroup,
         SCut,
         SCutGroup,
         topo,
         filled,
         topoMined,
         topoCheck,
         ModFactor):
         
    ## check variable inputs
    try:
        RCut, MCut, EBSBlockModel, RegBlockModel, ORR_ORSModel, topo, filled, topoMined, success = variableChecks(RCut = RCut,
                                                                                                                  MCut = MCut,
                                                                                                                  EBSBlockModel = EBSBlockModel ,
                                                                                                                  RegBlockModel = RegBlockModel,
                                                                                                                  ORR_ORSModel = ORR_ORSModel,
                                                                                                                  topo = topo,
                                                                                                                  topoMined = topoMined,
                                                                                                                  filled = filled)
    except:
        ValueError
        print("Program has stopped due to incomplete variable inputs")
        return None
    

        
    ## create database connection
    conn = databaseConnection(provider = "SQLOLEDB",
                              dataSource = "mpasdb",
                              dataBase = "MPAS_LOAD",
                              connection = "yes")
                   
    
    ## test EBS and RBM to see if it can be found
    topoEbs, topoRbm = checkTopo(EBS = EBSBlockModel,
                                 RBM = RegBlockModel,
                                 connection = conn)

    ## test to see topoEbs and topoRbm match up
    if topoEbs == False:
        print(f"No data found for EBS Block Model : {EBSBlockModel}, please check input for spelling mistakes")
    if topoRbm == False:
        print(f"No data found for Regularised Block Model : {RegBlockModel}, please check input for spelling mistakes")

    if topoEbs == topoRbm:
        topoCheck = "Identical"
    else:
        topoCheck = "Not Identical"
        print("WARNING: Topo of {EBSBlockModel} and {RegBlockModel} are not identical, program will proceed  if possible")

    
    ## Step 1 - The Base Line
    baseline = queryFunction(connection = conn,
                             step = "Baseline",
                             query = f"""
                                         SELECT mineralisation, sum(t_t)
                                         FROM {EBSBlockModel}
                                         WHERE fe_bin NOT IN('00.0','null')
                                         AND {topo}
                                         AND {filled}
                                         GROUP BY mineralisation """)
    
    ## Step 2 - The Lease loss
    leaseLoss = queryFunction(connection = conn,
                              step = "Lease Loss",
                              query = f"""SELECT lease_name, sum(t_t)
                                          FROM {EBSBlockModel}
                                          WHERE fe_bin NOT IN('00.0','null')
                                          AND {topo}
                                          AND {filled}
                                          AND lease Not in ('null')
                                          GROUP BY lease_name """)

    ## Step 3 - The Exrso loss
    exrsoLoss = queryFunction(connection = conn,
                              step = "Exrso loss",
                              query = f"""SELECT exrso_type, sum(t_t)
                                          FROM {EBSBlockModel}
                                          WHERE fe_bin NOT IN('00.0','null')
                                          AND {topo}
                                          AND {filled}
                                          AND lease Not in ('null')
                                          AND exrso = 1
                                          GROUP BY exrso_type """)

    ## Step 4 - The unclassified resource loss
    unclassifiedLoss = queryFunction(connection = conn,
                                     step = "Unclassified loss",
                                     query = f"""SELECT res_class, sum(t_t)
                                                 FROM {EBSBlockModel}
                                                 WHERE fe_bin NOT IN('00.0','null')
                                                 AND {topo}
                                                 AND {filled}
                                                 AND lease Not in ('null')
                                                 AND exrso = 0
                                                 AND res_class  in ('measured', 'indicated', 'inferred') 
                                                 GROUP BY res_class """)


    ## Step 5 - The Resource cutoff loss
    resCutoffLoss = queryFunction(connection = conn,
                                  step = "Resource cutoff loss",
                                  query = f"""SELECT {RCutGroup}, sum(t_t)
                                              FROM {EBSBlockModel}
                                              WHERE fe_bin NOT IN('00.0','null')
                                              AND {topo}
                                              AND {filled}
                                              AND lease Not in ('null')
                                              AND exrso = 0
                                              AND res_class  in ('measured', 'indicated', 'inferred') 
                                              AND {RCut}
                                              GROUP BY {RCutGroup} """)

    ## Step 6 - The mining cutoff
    miningCutoff = queryFunction(connection = conn,
                                 step = "Mining cutoff",
                                 query = f"""SELECT {MCutGroup}, sum(t_t)
                                             FROM {EBSBlockModel}
                                             WHERE fe_bin NOT IN('00.0','null')
                                             AND {topo}
                                             AND {filled}
                                             AND lease_name IS NOT NULL
                                             AND exrso = 0
                                             AND res_class  in ('measured', 'indicated', 'inferred') 
                                             AND {MCut}
                                             GROUP BY {MCutGroup} """)

    
    ## Step 7 - The regularisation cutoff
    regulisationCutoff = queryFunction(connection = conn,
                                       step = "regularisation cutoff",
                                       query = f"""SELECT {MCutGroup}, sum(t_t)
                                                   FROM {RegBlockModel}
                                                   WHERE fe_bin NOT IN('00.0','null')
                                                   AND {topo}
                                                   AND {filled}
                                                   AND lease_name IS NOT NULL
                                                   AND exrso = 0
                                                   AND res_class  in ('measured', 'indicated', 'inferred') 
                                                   AND {MCut}
                                                   GROUP BY {MCutGroup} """)

    ## Step 8 - Optimisation and Pit
    optimisationAndPut = queryFunction(connection = conn,
                                       step = "Optimisation and Pit",
                                       query = f"""SELECT {MCutGroup}, sum(t_t)
                                                   FROM {RegBlockModel}
                                                   WHERE fe_bin NOT IN ('00.0','null')
                                                   AND {topo}
                                                   AND {filled}
                                                   AND lease_name IS NOT NULL
                                                   AND exrso = 0
                                                   AND res_class  in ('measured', 'indicated', 'inferred') 
                                                   AND {MCut}
                                                   AND pit IN ('in')
                                                   GROUP BY {MCutGroup} """)
    ## Step 9 - Confidence
    Resclass = queryFunction(connection = conn,
                                       step = "Confidence",
                                       query = f"""SELECT {MCutGroup}, sum(t_t)
                                                   FROM {RegBlockModel}
                                                   WHERE 
                                                   fe_bin NOT IN('00.0','null')
                                                   AND {topo}
                                                   AND {filled}
                                                   AND lease NOT IN ('null')
                                                   AND exrso = 0
                                                   AND res_class  in ('measured', 'indicated') 
                                                   AND {MCut}
                                                   AND pit IN ('in')
                                                   GROUP BY {MCutGroup} """)

    ## Step 10 - finding the lease names and separating exploration lease
    explString = findLeaseName(leaseLoss)

    ## Step 11 - finding the loss for exploration and other lease
    explorationAndOtherLease = queryFunction(connection = conn,
                                             step = "Loss for exploration and other lease",
                                             query = f"""SELECT {MCutGroup}, sum(t_t)
                                                         FROM {RegBlockModel}
                                                         WHERE {mp_parcel}
                                                         AND {topo}
                                                         AND {filled}
                                                         AND lease NOT IN ('null')
                                                         AND exrso = 0
                                                         AND {MCut}
                                                         AND pit IN ('in')
                                                         AND res_class  in ('measured', 'indicated')
                                                         {explString}
                                                         GROUP BY {MCutGroup} """)

    ## Step 12 - The scheduling inventory - addition of other dest tag
    schedulingInventory = queryFunction(connection = conn,
                                        step = "scheduling inventory - addition of other dest tag",
                                        query = f"""SELECT {SCutGroup}, sum(t_t)
                                                    FROM {RegBlockModel}
                                                    WHERE {mp_parcel}
                                                    AND {topo}
                                                    AND {filled}
                                                    AND lease NOT IN ('null')
                                                    AND exrso = 0
                                                    AND {SCut}
                                                    AND pit IN ('in')
                                                    AND res_class  in ('measured', 'indicated')
                                                    {explString}
                                                    GROUP BY {SCutGroup} """)


    ## Step 13 - mined out loss
    minedOutLoss = queryFunction(connection = conn,
                                 step = "mined out loss",
                                 query = f"""SELECT res_class, sum(t_t)
                                             FROM {RegBlockModel}
                                             WHERE {mp_parcel}
                                             AND {topo}
                                             AND {filled}
                                             AND lease NOT IN ('null')
                                             AND exrso = 0
                                             AND {SCut}
                                             AND pit IN ('in')
                                             AND res_class  in ('measured', 'indicated')
                                             {explString}
                                             AND {topoMined}
                                             GROUP BY res_class""")

    ## Step 14a : ORR is NOT Empty 
    if ORR_ORSModel != "":

        ## 14A.1 ORR Approve Pit  
        ApprovedPit = queryFunction(connection = conn,
                                step = "oorApprovedPit",
                                query = f"""SELECT res_class, sum(t_t)
                                            FROM {ORR_ORSModel}
                                            WHERE {mp_parcel}
                                            AND {topo}
                                            AND {filled}
                                            AND lease Not in ('null')
                                            AND exrso = 0
                                            AND {SCut}
                                            AND pit IN ('in')
                                            AND res_class in ('measured', 'indicated'){explString}
                                            AND {topoMined}
                                            AND approved_pit = 1
                                            GROUP BY res_class """)
        
        ## 14A.2 ORR Scheduled Loss
        ScheduledOre = queryFunction(connection = conn,
                                step = "oorScheduledOre",
                                query = f"""SELECT res_class, sum(tsched_t)
                                            FROM {ORR_ORSModel}
                                            WHERE {mp_parcel}
                                            AND {topo}
                                            AND {filled}
                                            AND lease Not in ('null')
                                            AND exrso = 0
                                            AND rsv_dest in ('schedule_ore')
                                            AND pit IN ('in')
                                            AND res_class in ('measured', 'indicated'){explString}
                                            AND {topoMined}
                                            AND approved_pit = 1
                                            GROUP BY res_class """)

        ## 14A.3 ORR Processing Loss
        PlantLoss = queryFunction(connection = conn,
                                step = "oorPlantLoss",
                                query = f"""SELECT res_class, sum(s_f_t + s_l_t)
                                            FROM {ORR_ORSModel}
                                            WHERE {mp_parcel}
                                            AND {topo}
                                            AND {filled}
                                            AND lease Not in ('null')
                                            AND exrso = 0
                                            AND rsv_dest in ('schedule_ore')
                                            AND pit IN ('in')
                                            AND res_class in ('measured', 'indicated'){explString}
                                            AND {topoMined}
                                            AND approved_pit = 1
                                            GROUP BY res_class """)

## MISSing VARIABLE t_fe>= 50
        ## 14A.4 ORR Jorc Class
        JorcClassDF = queryFunction(connection = conn,
                                step = "oorJorcClass",
                                query = f"""SELECT res_class, sum(s_f_t + s_l_t)
                                            FROM {ORR_ORSModel}
                                            WHERE {mp_parcel}
                                            AND {topo}
                                            AND {filled}
                                            AND lease Not in ('null')
                                            AND exrso = 0
                                            AND rsv_dest in ('schedule_ore')
                                            AND pit IN ('in')
                                            AND res_class in ('measured', 'indicated'){explString}
                                            AND {topoMined}
                                            AND approved_pit = 1
                                            AND rsv_class_jorc IN ('proved', 'probable')
                                            GROUP BY res_class """)
    
    else:

        ## Step 14 b
        ## RBM Approved Pit #### Varibale MISSING
        ApprovedPit = queryFunction(connection = conn,
                                step = "ApprovedPit",
                                query = f"""SELECT res_class, sum(t_t)
                                            FROM  {RegBlockModel}
                                            WHERE {mp_parcel}
                                            AND {topo}
                                            AND {filled}
                                            AND lease Not in ('null')
                                            AND exrso = 0
                                            AND {SCut}
                                            AND pit IN ('in')
                                            AND res_class in ('measured', 'indicated'){explString}
                                            AND {topoMined}
                                            AND approved_pit = 1
                                            GROUP BY res_class """)

        indicated_ore = 0
        measured_ore = 0
        indicated_plant = 0
        measured_plant = 0
        inut = 'mp_dest' 

        for i in range(0,len(ModFactor)):
            Field = ModFactor['Field'][i]
            plant = ModFactor['plant'][i]
            Ratio = ModFactor['Ratio'][i]
            if i == 0:
                if Ratio != 0 :
                    
                    FieldList = f"'{str(Field)}'"
            else:
                if Ratio != 0 :
                
                    FieldList += f", '{str(Field)}'" # dealing with inut inputs for Field List Colum of ModFactor Table

        ScheduleAndPlantLoss = queryFunction(connection = conn,
                    step = "ScheduleAndPlantLoss",
                    query = f"""SELECT res_class, sum(t_t), sum({plant}) 
                                FROM {RegBlockModel}
                                WHERE {mp_parcel}
                                AND {topo}
                                AND lease Not in ('null')
                                AND exrso = 0
                                AND {SCut}
                                AND pit IN ('in')
                                AND res_class in ('measured', 'indicated'){explString}
                                AND {topoMined}
                                AND approved_pit = 1
                                AND filled = 999999
                                AND {inut} IN ({FieldList})  
                                GROUP BY res_class """) 

        Ratio = ModFactor[ModFactor['Ratio'] != 0]['Ratio'].mean()
        if ScheduleAndPlantLoss.iloc[0, 0] == 'indicated':
            indicated_ore = indicated_ore + ScheduleAndPlantLoss.iloc[0, 1] * Ratio
            indicated_plant = indicated_plant + ScheduleAndPlantLoss.iloc[0, 2] * Ratio
        else:
            measured_ore = measured_ore +  ScheduleAndPlantLoss.iloc[0, 1]* Ratio
            measured_plant = measured_plant +  ScheduleAndPlantLoss.iloc[0, 2] * Ratio    

        if  ScheduleAndPlantLoss.iloc[0, 0] != 'indicated': 
            indicated_ore = indicated_ore + ScheduleAndPlantLoss.iloc[1, 1] * Ratio
            indicated_plant = indicated_plant + ScheduleAndPlantLoss.iloc[1, 2] * Ratio
        else:
            if len(ScheduleAndPlantLoss) > 1:
                    measured_ore = measured_ore +  ScheduleAndPlantLoss.iloc[1, 1]* Ratio
                    measured_plant = measured_plant +  ScheduleAndPlantLoss.iloc[1, 2] * Ratio  
            else:
                    measured_ore = measured_ore
                    measured_plant = measured_plant 
        
        ScheduledOre = pd.DataFrame([indicated_ore, measured_ore], ['inidcated_ore', 'measured_ore']).reset_index().rename(columns = {'index': 'ScheduleLoss', 0 : 'SLvalue'})
        PlantLoss = pd.DataFrame([indicated_plant, measured_plant], ['inidcated_plant', 'measured_plant']).reset_index().rename(columns = {'index': 'ProcessingLoss', 0 : 'PLvalue'})


    ## Jorc Class
            ## Jorc Class
        jorc_indicated_ore = 0
        jorc_measured_ore = 0
        jorc_indicated_plant = 0
        jorc_measured_plant = 0
        jorc_inut = 'mp_dest' 

        for i in range(0,len(ModFactor)):
            Field = ModFactor['Field'][i]
            plant = ModFactor['plant'][i]
            Ratio = ModFactor['Ratio'][i]
            if i == 0:
                if Ratio != 0 :
                    
                    FieldList = f"'{str(Field)}'"
            else:
                if Ratio != 0 :
                
                    FieldList += f", '{str(Field)}'" # dealing with inut inputs for Field List Colum of ModFactor Table

        JorcClass = queryFunction(connection = conn,
                    step = "JorcClass",
                    query = f"""SELECT res_class, sum({plant})
                                FROM {RegBlockModel}
                                WHERE {mp_parcel}
                                AND {topo}
                                AND  filled = 999999
                                AND lease Not in ('null')
                                AND exrso = 0
                                AND {SCut}
                                AND pit IN ('in')
                                AND res_class in ('measured', 'indicated'){explString}
                                AND {topoMined}
                                AND approved_pit = 1
                                AND {inut} IN ({FieldList})
                                AND rsv_class_jorc IN('proved', 'probable')
                                GROUP BY res_class """)

        Ratio = ModFactor[ModFactor['Ratio'] != 0]['Ratio'].mean()
        if JorcClass.iloc[0, 0] == 'indicated':
            jorc_indicated_plant = jorc_indicated_plant + JorcClass.iloc[0, 1] * Ratio
        else:
            jorc_measured_plant = jorc_measured_plant + JorcClass.iloc[0, 1] * Ratio    

        if  JorcClass.iloc[0, 0] != 'indicated': 
            jorc_indicated_plant = jorc_indicated_plant + JorcClass.iloc[1, 1] * Ratio
        else:
            if len(ScheduleAndPlantLoss) > 1:
                jorc_measured_plant = jorc_measured_plant +  JorcClass.iloc[1, 1] * Ratio
            else:
                    jorc_measured_plant = jorc_measured_plant

        JorcClassDF = pd.DataFrame([jorc_indicated_plant, jorc_measured_plant], ['inidcated','measured']).reset_index().rename(columns = {'index': 'JorcClass', 0 : 'JCvalue'})
        

    return baseline, leaseLoss, exrsoLoss, unclassifiedLoss, resCutoffLoss, miningCutoff, regulisationCutoff,optimisationAndPut, Resclass, explorationAndOtherLease, schedulingInventory, minedOutLoss, ApprovedPit,ScheduledOre, PlantLoss,  JorcClassDF



baseline, leaseLoss, exrsoLoss, unclassifiedLoss, resCutoffLoss, miningCutoff,regulisationCutoff, optimisationAndPut,Resclass, explorationAndOtherLease, schedulingInventory, minedOutLoss,ApprovedPit,ScheduledOre, PlantLoss, JorcClassDF  = main(EBSBlockModel,
            RegBlockModel,
            ORR_ORSModel,
            RCut,
            RCutGroup,
            MCut,
            MCutGroup,
            SCut,
            SCutGroup,
            topo,
            filled,
            topoMined,
            topoCheck,
            ModFactor)


Creating database connection
Connection Successfull
Running query: 
                SELECT max(topo) from waebs_DepA_20220928_002
                WHERE topo != 999999
                 


C:\Users\mike.heck\AppData\Local\Temp\ipykernel_25692\3156076022.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Running query: 
                SELECT max(topo) from waapr2_DepA_20220928_021
                WHERE topo != 999999
                 


C:\Users\mike.heck\AppData\Local\Temp\ipykernel_25692\3156076022.py:57: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.







**************************************************
Running Baseline query:
SELECT mineralisation, sum(t_t)
FROM waebs_DepA_20220928_002
WHERE fe_bin NOT IN('00.0','null')
AND topo>202208
AND filled = 999999
GROUP BY mineralisation 
**************************************************


...

ApprovedPit query complete
ApprovedPit query returned a non-empty dataset, program will continue




**************************************************
Running ScheduleAndPlantLoss query:
SELECT res_class, sum(t_t), sum(s_l_t + s_f_t) 
FROM waapr2_DepA_20220928_021
WHERE t_fe>50
AND topo>202208
AND lease Not in ('null')
AND exrso = 0
AND t_fe>=58
AND pit IN ('in')
AND res_class in ('measured', 'indicated')AND lease_name NOT IN ('aml70/00248_sec_071', 'aml70/00248_sec_072', 'aml70/00248_sec_077', 'notlive')
AND topo>99998
AND approved_pit = 1
AND filled = 999999
AND mp_dest IN ('hg', 'hga')  
GROUP BY res_class 
**************************************************


C:\Users\mike.heck\AppData\Local\Temp\ipykernel_25692\3156076022.py:144: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



ScheduleAndPlantLoss query complete
ScheduleAndPlantLoss query returned a non-empty dataset, program will continue




**************************************************
Running JorcClass query:
SELECT res_class, sum(s_l_t + s_f_t)
FROM waapr2_DepA_20220928_021
WHERE t_fe>50
AND topo>202208
AND  filled = 999999
AND lease Not in ('null')
AND exrso = 0
AND t_fe>=58
AND pit IN ('in')
AND res_class in ('measured', 'indicated')AND lease_name NOT IN ('aml70/00248_sec_071', 'aml70/00248_sec_072', 'aml70/00248_sec_077', 'notlive')
AND topo>99998
AND approved_pit = 1
AND mp_dest IN ('hg', 'hga')
AND rsv_class_jorc IN('proved', 'probable')
GROUP BY res_class 
**************************************************


C:\Users\mike.heck\AppData\Local\Temp\ipykernel_25692\3156076022.py:144: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



JorcClass query complete
JorcClass query returned a non-empty dataset, program will continue


FORMATTING OUTPUT DATA

In [21]:

def rename_columns(baseline, leaseLoss, exrsoLoss, unclassifiedLoss, resCutoffLoss, miningCutoff,regulisationCutoff, optimisationAndPut,Resclass, explorationAndOtherLease, schedulingInventory, minedOutLoss,ApprovedPit,ScheduledOre, PlantLoss, JorcClassDF):
    baseline = baseline.rename(columns = {'mineralisation': 'baseline', '': 'BSvalue'})
    leaseLoss = leaseLoss.rename(columns = {'mineralisation': 'baseline', '': 'LLvalue'})
    exrsoLoss = exrsoLoss.rename(columns = {'exrso_type': 'EXRSO', '': 'EXvalue'})
    unclassifiedLoss = unclassifiedLoss.rename(columns = {'res_class': 'unclassified_resource', '': 'URvalue'})
    resCutoffLoss = resCutoffLoss.rename(columns = {'dest_tag': 'resource_cutoff_loss', '': 'RCvalue'})
    miningCutoff = miningCutoff.rename(columns = {'mp_dest': 'mininng_cutoff_loss', '': 'MCvalue'})
    regulisationCutoff = regulisationCutoff.rename(columns = {'mp_dest': 'regularisation_cutoff', '': 'REGvalue'}) ## aka SMU Cut off
    optimisationAndPut = optimisationAndPut.rename(columns = {'mp_dest': 'optimisation_and_pit', '': 'OPvalue'})
    Resclass = Resclass.rename(columns = {'mp_dest': 'Resclass', '': 'ResClassvalue'})
    explorationAndOtherLease = explorationAndOtherLease.rename(columns = {'mp_dest': 'explorationAndOtherLease', '': 'ELvalue'})
    schedulingInventory = schedulingInventory.rename(columns = {'mp_dest': 'SchedulingInventory', '': 'SIvalue'}) # aka Potential VCOG
    minedOutLoss = minedOutLoss.rename(columns = {'res_class': 'minedOutLoss', '': 'MOLvalue'})
    ApprovedPit = ApprovedPit.rename(columns = {'res_class': 'ApprovedPit', '': 'APvalue'})
    ScheduledOre = ScheduledOre.rename(columns = {'res_class': 'ScheduleLoss', '': 'SLvalue'})
    PlantLoss = PlantLoss.rename(columns = {'res_class': 'ScheduleLoss', '': 'PLvalue'})
    JorcClassDF = JorcClassDF.rename(columns = {'res_class': 'ScheduleLoss', '': 'JCvalue'})

    #Create table
    table = pd.concat([baseline, leaseLoss, exrsoLoss, unclassifiedLoss, resCutoffLoss, miningCutoff, regulisationCutoff, optimisationAndPut,Resclass, explorationAndOtherLease, schedulingInventory, minedOutLoss, ApprovedPit,ScheduledOre, PlantLoss, JorcClassDF ], axis = 1)
    return table 
table = rename_columns(baseline, leaseLoss, exrsoLoss, unclassifiedLoss, resCutoffLoss, miningCutoff,regulisationCutoff, optimisationAndPut,Resclass, explorationAndOtherLease, schedulingInventory, minedOutLoss,ApprovedPit,ScheduledOre, PlantLoss, JorcClassDF)

In [22]:
table

,baseline,BSvalue,lease_name,LLvalue,EXRSO,EXvalue,unclassified_resource,URvalue,resource_cutoff_loss,RCvalue,...,minedOutLoss,MOLvalue,ApprovedPit,APvalue,ScheduleLoss,SLvalue,ProcessingLoss,PLvalue,JorcClass,JCvalue
0,1.0,2.769226e+07,aml70/00248_sec_071,8.531534e+07,exrsofilled,5.719659e+07,indicated,1.072231e+07,hg,3.375012e+07,...,indicated,228710.299805,indicated,228710.299805,inidcated_ore,228710.299805,inidcated_plant,228710.300049,inidcated,228710.300049
1,2.0,6.244465e+07,aml70/00248_sec_072,9.031174e+07,exrsvfilled,5.927663e+07,inferred,3.331846e+06,NaN,NaN,...,NaN,NaN,NaN,NaN,measured_ore,0.000000,measured_plant,0.000000,measured,0.000000
2,5.0,9.607645e+07,aml70/00248_sec_077,6.423203e+06,exrsvremnant,2.715166e+05,measured,5.541447e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,ml248sa_sec071,4.163020e+06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,notlive,6.664822e+01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


"REMAINING" & "LOSS" calculations

In [23]:
summary = pd.DataFrame(table.sum(axis = 0).iloc[1:]).transpose()
labels = pd.DataFrame(pd.Series(['Remaining', 'Loss']))
summary = pd.concat((labels,summary),axis=1).fillna('')

C:\Users\mike.heck\AppData\Local\Temp\ipykernel_25692\3407683809.py:1: FutureWarning:

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



In [24]:
summary

,0,BSvalue,lease_name,LLvalue,EXvalue,URvalue,RCvalue,MCvalue,REGvalue,OPvalue,ResClassvalue,ELvalue,SIvalue,MOLvalue,APvalue,SLvalue,PLvalue,JCvalue
0,Remaining,186213363.773615,aml70/00248_sec_071aml70/00248_sec_072aml70/00...,186213363.773615,116744735.842821,69468627.930794,33750120.228172,33750120.228172,24263174.5271,6459034.068359,6459034.068359,228710.299805,228710.299805,228710.299805,228710.299805,228710.299805,228710.300049,228710.300049
1,Loss,,,,,,,,,,,,,,,,,


In [25]:
#Loss Calculations
summary['LLvalue'].iloc[1] = summary['BSvalue'].iloc[0] - summary['LLvalue'].iloc[0]
summary['EXvalue'].iloc[1] = summary['EXvalue'].iloc[0] * -1
if summary['EXvalue'][1] == '':
    summary['EXvalue'].iloc[0] = 0
else:
    summary['EXvalue'].iloc[0] = summary['EXvalue'].iloc[1] + summary['LLvalue'].iloc[0] # remaining Value

summary['URvalue'].iloc[1] = summary['EXvalue'].iloc[0] - summary['URvalue'].iloc[0]
summary['RCvalue'].iloc[1] = summary['URvalue'].iloc[0] - summary['RCvalue'].iloc[0]
summary['MCvalue'].iloc[1] = summary['RCvalue'].iloc[0] - summary['MCvalue'].iloc[0]
summary['REGvalue'].iloc[1] = summary['MCvalue'].iloc[0] - summary['REGvalue'].iloc[0]
summary['OPvalue'].iloc[1] = summary['REGvalue'].iloc[0] - summary['OPvalue'].iloc[0]
summary['ResClassvalue'].iloc[1] = summary['OPvalue'].iloc[0] - summary['ResClassvalue'].iloc[0]
summary['ELvalue'].iloc[1] = summary['ResClassvalue'].iloc[0] - summary['ELvalue'].iloc[0]
summary['SIvalue'].iloc[1] = -1*(summary['ELvalue'].iloc[0] - summary['SIvalue'].iloc[0])
summary['MOLvalue'].iloc[1] = summary['SIvalue'].iloc[0] - summary['MOLvalue'].iloc[0]

summary['APvalue'].iloc[1] = summary['MOLvalue'].iloc[0] - summary['APvalue'].iloc[0]
summary['SLvalue'].iloc[1] = summary['APvalue'].iloc[0] - summary['SLvalue'].iloc[0]
summary['PLvalue'].iloc[1] = summary['SLvalue'].iloc[0] - summary['PLvalue'].iloc[0]
summary['JCvalue'].iloc[1] = summary['PLvalue'].iloc[0] - summary['JCvalue'].iloc[0]

## Waterfall Chart

In [26]:
summary

,0,BSvalue,lease_name,LLvalue,EXvalue,URvalue,RCvalue,MCvalue,REGvalue,OPvalue,ResClassvalue,ELvalue,SIvalue,MOLvalue,APvalue,SLvalue,PLvalue,JCvalue
0,Remaining,186213363.773615,aml70/00248_sec_071aml70/00248_sec_072aml70/00...,186213363.773615,69468627.930794,69468627.930794,33750120.228172,33750120.228172,24263174.5271,6459034.068359,6459034.068359,228710.299805,228710.299805,228710.299805,228710.299805,228710.299805,228710.300049,228710.300049
1,Loss,,,0.0,-116744735.842821,0.0,35718507.702621,0.0,9486945.701073,17804140.45874,0.0,6230323.768555,-0.0,0.0,0.0,0.0,-0.000244,0.0


In [27]:
# Waterfall Chart Calculations
lease = summary['BSvalue'][0] - summary['LLvalue'][0]
exrso = table[table['EXRSO'].str.contains('exrso', na = False)]['EXvalue'].sum()
erxsv = table[table['EXRSO'].str.contains('exrsv', case = False, na = False)]['EXvalue'].sum()
UnclassMI = summary['LLvalue'][0] - exrso - erxsv - summary['URvalue'][0]
ResCutoff = summary['URvalue'][0] - summary['RCvalue'][0] 

# Exploration Value Calc
summary['MCvalue'] = np.where(summary['MCvalue'] == '', 0, summary['MCvalue'])
summary['REGvalue'] = np.where(summary['REGvalue'] == '', 0, summary['REGvalue'])

if summary['MCvalue'][1]> 0:
    exploration = (summary['RCvalue'][0] - summary['MCvalue'][1]) - abs(summary['REGvalue'][1]) - summary['OPvalue'][1] - summary['ResClassvalue'][1] - summary['ELvalue'][0]
else:
    exploration = summary['RCvalue'][0]- abs(summary['REGvalue'][1]) - summary['OPvalue'][1] - summary['ResClassvalue'][1] - summary['ELvalue'][0]

LOMP = summary['ELvalue'][0] + summary['SIvalue'][1] - summary['MOLvalue'][1] - adjustment_2
OreReserve = LOMP - summary['APvalue'][1] - summary['SLvalue'][1] - summary['PLvalue'][1] - summary['JCvalue'][1] - adjustment_3 

# Waterfall title:
title = EBSBlockModel.split('_')[1].upper() +'_' + str(date.today().year)


In [28]:
waterfall = []
waterfall = [
                summary['BSvalue'][0],          # Model Mineralized
                -lease,                         
                -exrso,
                -erxsv,
                -UnclassMI,                     # Unclassified (MI)
                -ResCutoff,                     # Resource Cutoff (MI)
                adjustment_1,                   # Adjustment 1
                -(summary['RCvalue'][0] - adjustment_1),         # Mineral Resource (incl Reservce)
                -abs(summary['MCvalue'][1]),    # Mining Cutoff
                -abs(summary['REGvalue'][1]),   # SMU / Regularisation
                -summary['OPvalue'][1],         # Optimisation loss
                -summary['ResClassvalue'][1],   # Res Class
                -exploration,
                summary['SIvalue'][1],          # Potential VCOG
                -summary['MOLvalue'][1],        # Mined Out Loss
                adjustment_2,                   # Adjustment 2 
                LOMP,                           # SI * (LOMP Inventory)
                -summary['APvalue'][1],         # Approved Pit
                -summary['SLvalue'][1],         # Schedule Loss
                -summary['PLvalue'][1],         # Processing Loss
                -summary['JCvalue'][1],         # Jorc Class
                adjustment_3,                   # Adjustment 3
                -OreReserve,                    # ore reserve
]

In [29]:
#Rounding  Formatting
waterfall=[round((x/1000000)) for x in waterfall]

In [30]:
fig = go.Figure(go.Waterfall(
    orientation = "v",
    measure = ["relative", "relative", "relative","relative","relative","relative","relative", "total","relative","relative","relative","relative","relative","relative","relative","relative", "total","relative","relative","relative","relative","relative", "total" ],

    x = ["Model Mineralised", "Lease", "EXRSO", "EXRSV", "Unclassified (MI)", " Resource Cutoff (MI)", "Adjustment1", "Mineral Resource (Inc. REserve)", "Mining Cutoff", "SMU/ Regularisation", "Optimisation & Pit Design", "Res Class (inferred)", "Exploration Lease", "Potential VCOG", "Mined Out", "Adjustment2", "SI* (LOMP Inventory)", "Approved Pit", "Schedule Loss", "Processing Loss", "Jorc Class", "Adjustment3", "Ore Reserve"],

    textposition = "outside",
    text = [abs(x) for x in waterfall],
    y =  waterfall,
    connector = {"visible": False},
    base = 0,

    decreasing = {"marker":{"color":"Red"}},
    increasing = {"marker":{"color":"Green"}},
    totals = {"marker":{"color":"Blue"}}
))

fig.update_layout(title = f'{title} Waterfall Chart',  waterfallgap  = 0.1,
        autosize=False,
        width=2250,
        height=1100,
        font=dict(
            size=25,  # Set the font size here)
        ))

fig.update_xaxes(tickangle=-90)

fig.show()

Saving Waterfall Chart

In [31]:
#Save file
if not os.path.exists("..\Waterfall_Plots"):
    os.mkdir("..\Waterfall_Plots")

fig.write_image(f"..\Waterfall_Plots/{title}_Waterfall_plot.png")

Save CSV File

In [32]:
#Save file
if not os.path.exists("..\csv_files"):
    os.mkdir("..\csv_files")

summary.to_csv(f"..\csv_files/{title}_Remaining&Loss.csv")
table.fillna("").to_csv(f"..\csv_files/{title}_RawValues.csv")